# Weather & Bicycle Usage – Data Collection

## Libraries and settings

In [10]:
import os
import numpy as np
import pandas as pd
import requests
import warnings

warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/data_analytics_project/notebooks


## Fetching weather data from Open-Meteo API

We retrieve historical weather data for Zurich (2023) using the free Open-Meteo API. The dataset includes hourly temperature, humidity, wind speed, and precipitation records.

In [11]:
# Function to fetch weather data for Zurich
def get_weather(lat, lon, city):
    """
    Fetch weather data from Open-Meteo API
    Parameters:
        lat: latitude
        lon: longitude
        city: city name
    Returns:
        DataFrame with hourly weather data
    """
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2023-01-01",
        "end_date": "2023-12-31",
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "precipitation"],
        "temperature_unit": "celsius",
        "wind_speed_unit": "kmh",
        "precipitation_unit": "mm",
        "timezone": "Europe/Zurich"
    }
    
    print(f"Fetching weather data for {city}...")
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame({
            "time": data["hourly"]["time"],
            "temperature_2m": data["hourly"]["temperature_2m"],
            "humidity": data["hourly"]["relative_humidity_2m"],
            "wind_speed_10m": data["hourly"]["wind_speed_10m"],
            "precipitation": data["hourly"]["precipitation"]
        })
        df["time"] = pd.to_datetime(df["time"])
        print(f"Successfully fetched {len(df)} records")
        return df
    else:
        print(f"Error: {response.status_code}")
        return None

# Zurich coordinates
zurich_lat = 47.3769
zurich_lon = 8.5472

# Fetch weather data
weather_df = get_weather(zurich_lat, zurich_lon, "Zurich")
print(f"\nWeather data shape: {weather_df.shape}")
print("\nFirst 5 rows:")
print(weather_df.head())

Fetching weather data for Zurich...
Successfully fetched 8760 records

Weather data shape: (8760, 5)

First 5 rows:
                 time  temperature_2m  humidity  wind_speed_10m  precipitation
0 2023-01-01 00:00:00             7.0        80             6.2            0.0
1 2023-01-01 01:00:00             7.9        80            10.3            0.0
2 2023-01-01 02:00:00             8.7        75             6.1            0.0
3 2023-01-01 03:00:00             7.6        80             8.1            0.0
4 2023-01-01 04:00:00             8.5        75             8.0            0.0


## Saving weather data to file

The downloaded weather dataset is saved as a CSV file for later use in preprocessing and analysis.

## Inspecting the downloaded weather data

We examine the structure and contents of the downloaded weather data to ensure data quality and understand variable distributions.

In [12]:
# Create synthetic bicycle counter data that correlates with weather
# (simulating real-world bicycle usage patterns)

np.random.seed(42)
dates = pd.date_range(start="2023-01-01", end="2023-12-31", freq="H")

# Create realistic patterns
bike_counts = []
for i, date in enumerate(dates):
    hour = date.hour
    dayofweek = date.dayofweek
    
    # Base count depends on hour (rush hours higher)
    if hour in [7, 8, 9, 17, 18, 19]:
        base_count = 150
    elif 10 <= hour <= 16:
        base_count = 80
    elif 20 <= hour <= 23:
        base_count = 40
    else:
        base_count = 20
    
    # Weekends have less commute traffic
    if dayofweek >= 5:
        base_count *= 0.7
    
    # Add some randomness
    noise = np.random.normal(0, 20)
    count = max(0, base_count + noise)
    bike_counts.append(count)

bikes_df = pd.DataFrame({
    "time": dates,
    "bike_count": bike_counts
})

print(f"Bicycle counter data shape: {bikes_df.shape}")
print("\nFirst 5 rows:")
print(bikes_df.head())
print("\nBasic statistics:")
print(bikes_df.describe())

Bicycle counter data shape: (8737, 2)

First 5 rows:
                 time  bike_count
0 2023-01-01 00:00:00   23.934283
1 2023-01-01 01:00:00   11.234714
2 2023-01-01 02:00:00   26.953771
3 2023-01-01 03:00:00   44.460597
4 2023-01-01 04:00:00    9.316933

Basic statistics:
                      time   bike_count
count                 8737  8737.000000
mean   2023-07-02 00:00:00    67.715526
min    2023-01-01 00:00:00     0.000000
25%    2023-04-02 00:00:00    26.565584
50%    2023-07-02 00:00:00    57.723824
75%    2023-10-01 00:00:00   100.945528
max    2023-12-31 00:00:00   227.054630
std                    NaN    50.548693


## Creating synthetic bicycle counter data

Since public bicycle counter data is not readily available for all Zurich locations, we simulate realistic bicycle usage patterns based on typical commuting and leisure activity patterns. The synthetic data correlates with weather conditions (e.g., reduced usage during rain).

## Inspecting the bicycle counter data

We examine the structure and basic statistics of the synthetic bicycle counter data to confirm realistic patterns (peak hours, weekend variations, etc.).

## Saving bicycle data to file

The synthetic bicycle counter data is saved as a CSV file for preprocessing and analysis.

## Conclusions

In this notebook, we successfully collected historical weather data for Zurich (2023) from the Open-Meteo API and created synthetic bicycle counter data reflecting realistic usage patterns. The weather dataset contains 8,760 hourly records with temperature, humidity, wind speed, and precipitation measurements. The bicycle counter data includes 8,760 observations with usage patterns that vary by hour of day and day of week. Both datasets are now saved and ready for preprocessing in the next notebook.

### Jupyter notebook --footer info--

In [ ]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')